In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tqdm.notebook import tqdm
from keras.utils import to_categorical
import gensim
import numpy as np
from keras.callbacks import callbacks
from sentence_transformers import SentenceTransformer
from sklearn.metrics import f1_score
import re
from sentence_transformers import SentenceTransformer

Using TensorFlow backend.
/home/nikamanth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nikamanth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nikamanth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nikamanth/.local/lib/python3.7/site-packages/tenso

In [2]:
def write_result(file_name, preds):
    with open('../results/'+file_name, 'wt') as file:
        for i in preds:
            file.write(i+'\n')
    with open('../results/bert_result.txt', 'wt') as file:
        for i in preds:
            file.write(i+'\n')
    !python ../NADI-2020_release_1.0/NADI_release/NADI-DID-Scorer.py ../tsv/gold1.txt ../results/bert_result.txt

def write_result2(file_name, preds):
    with open('../results/'+file_name, 'wt') as file:
        for i in preds:
            file.write(i+'\n')
    with open('../results/bert_result.txt', 'wt') as file:
        for i in preds:
            file.write(i+'\n')
    !python ../NADI-2020_release_1.0/NADI_release/NADI-DID-Scorer.py ../tsv/gold2.txt ../results/bert_result.txt

def preprocess_text(train_list, test_list):
    X_train_corrected_tweets = []
    for tweet in tqdm(train_list):
        new_tweet = re.findall( '[^A-Za-z:/_.0-9\\#@,=+\(\)]+' ,tweet)
        new_tweet = " ".join(new_tweet).replace('\xa0','').replace('\u200c','').replace('\U000fe329','').replace('\u2066','').replace('\u2069','').strip()
        X_train_corrected_tweets.append(new_tweet)

    X_dev_corrected_tweets = []
    for tweet in tqdm(test_list):
        new_tweet = re.findall( '[^A-Za-z:/_.0-9\\#@,=+\(\)]+' ,tweet) #[^\x00-\x19\x21-\x7F]+
        new_tweet = " ".join(new_tweet).replace('\xa0','').replace('\u200c','').replace('\U000fe329','').replace('\u2066','').replace('\u2069','').strip()
        X_dev_corrected_tweets.append(new_tweet)
    return X_train_corrected_tweets, X_dev_corrected_tweets

In [3]:
#bert - multilingual
bert = SentenceTransformer('distiluse-base-multilingual-cased')

# task1

# unbalanced

In [4]:
train_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/train_labeled.tsv',sep='\t')
dev_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv',sep='\t')

X_train_original,y_train_original = train_df["#2 tweet_content"],train_df["#3 country_label"]
X_dev_original,y_dev_original = dev_df["#2 tweet_content"],dev_df["#3 country_label"]

In [5]:
labels = {}
uni = y_train_original.unique()
for i in range(len(uni)):
    labels[uni[i]] = i

y_train_index = [labels[i] for i in y_train_original]
y_dev_index = [labels[i] for i in y_dev_original]

y_train = to_categorical(y_train_index, num_classes=21)
y_dev = to_categorical(y_dev_index, num_classes=21)

In [6]:
X_train_corrected, X_dev_corrected = preprocess_text(X_train_original, X_dev_original)
X_train = bert.encode(X_train_corrected,batch_size=32,show_progress_bar=True)
X_dev = bert.encode(X_dev_corrected,batch_size=32,show_progress_bar=True)
X_train = np.vstack(X_train)
X_dev = np.vstack(X_dev)

Batches:   0%|          | 0/657 [00:00<?, ?it/s]

Batches: 100%|██████████| 155/155 [00:57<00:00,  2.71it/s]


In [17]:
model = Sequential()
model.add(Dense(512, activation='relu'))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=50,validation_data=(X_dev,y_dev),
                   callbacks=[callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                                      patience=1, verbose=0, mode='auto', 
                                                      baseline=None, restore_best_weights=True)])
prediction = model.predict_classes(X_dev)

Train on 21000 samples, validate on 4957 samples
Epoch 1/50
21000/21000 [==============================] - 2s 115us/step - loss: 2.5219 - accuracy: 0.2566 - val_loss: 2.4410 - val_accuracy: 0.2855
Epoch 2/50
21000/21000 [==============================] - 2s 109us/step - loss: 2.3499 - accuracy: 0.3030 - val_loss: 2.4284 - val_accuracy: 0.2957
Epoch 3/50
21000/21000 [==============================] - 2s 109us/step - loss: 2.2623 - accuracy: 0.3258 - val_loss: 2.4256 - val_accuracy: 0.2951
Epoch 4/50
21000/21000 [==============================] - 2s 109us/step - loss: 2.1835 - accuracy: 0.3427 - val_loss: 2.4552 - val_accuracy: 0.2855


In [18]:
reverse_label_map = {value : key for (key, value) in labels.items()}
print(reverse_label_map)
pred = model.predict_classes(X_dev)
pred = [reverse_label_map[i] for i in pred]
# print(pred)
len(pred)
write_result('bert_task1_unbal.txt',pred)

{0: 'Iraq', 1: 'Egypt', 2: 'Morocco', 3: 'Libya', 4: 'United_Arab_Emirates', 5: 'Mauritania', 6: 'Saudi_Arabia', 7: 'Bahrain', 8: 'Syria', 9: 'Djibouti', 10: 'Lebanon', 11: 'Oman', 12: 'Palestine', 13: 'Algeria', 14: 'Somalia', 15: 'Jordan', 16: 'Tunisia', 17: 'Kuwait', 18: 'Yemen', 19: 'Sudan', 20: 'Qatar'}
/home/nikamanth/anaconda3/envs/torch/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 15.56 %
MACRO AVERAGE RECALL SCORE: 11.60 %
MACRO AVERAGE F1 SCORE: 11.13 %
OVERALL ACCURACY: 29.51 %



# balanced

In [4]:
train_df = pd.read_csv('../tsv/oversample_train.tsv',sep='\t')
dev_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv',sep='\t')

X_train_original,y_train_original = train_df["#2 tweet_content"],train_df["#3 country_label"]
X_dev_original,y_dev_original = dev_df["#2 tweet_content"],dev_df["#3 country_label"]

In [5]:
labels = {}
uni = y_train_original.unique()
for i in range(len(uni)):
    labels[uni[i]] = i

y_train_index = [labels[i] for i in y_train_original]
y_dev_index = [labels[i] for i in y_dev_original]

y_train = to_categorical(y_train_index, num_classes=21)
y_dev = to_categorical(y_dev_index, num_classes=21)

In [6]:
X_train_corrected, X_dev_corrected = preprocess_text(X_train_original, X_dev_original)
X_train = bert.encode(X_train_corrected,batch_size=32,show_progress_bar=True)
X_dev = bert.encode(X_dev_corrected,batch_size=32,show_progress_bar=True)
X_train = np.vstack(X_train)
X_dev = np.vstack(X_dev)

Batches:   0%|          | 2/2936 [00:00<03:16, 14.91it/s]

Batches: 100%|██████████| 155/155 [00:58<00:00,  2.63it/s]


In [7]:
model = Sequential()
model.add(Dense(512, activation='relu'))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=50,validation_data=(X_dev,y_dev),
                   callbacks=[callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                                      patience=2, verbose=0, mode='auto', 
                                                      baseline=None, restore_best_weights=True)])
prediction = model.predict_classes(X_dev)

Train on 93933 samples, validate on 4957 samples
Epoch 1/50
93933/93933 [==============================] - 10s 109us/step - loss: 2.2775 - accuracy: 0.3423 - val_loss: 2.8395 - val_accuracy: 0.1455
Epoch 2/50
93933/93933 [==============================] - 10s 105us/step - loss: 1.5558 - accuracy: 0.5585 - val_loss: 2.9117 - val_accuracy: 0.1582
Epoch 3/50
93933/93933 [==============================] - 10s 106us/step - loss: 1.1250 - accuracy: 0.6880 - val_loss: 3.0205 - val_accuracy: 0.1810


In [8]:
reverse_label_map = {value : key for (key, value) in labels.items()}
print(reverse_label_map)
pred = model.predict_classes(X_dev)
pred = [reverse_label_map[i] for i in pred]
# print(pred)
len(pred)
write_result('bert_task1_bal.txt',pred)

{0: 'Lebanon', 1: 'Sudan', 2: 'Djibouti', 3: 'Saudi_Arabia', 4: 'Oman', 5: 'Yemen', 6: 'Bahrain', 7: 'Mauritania', 8: 'Kuwait', 9: 'Iraq', 10: 'Palestine', 11: 'Jordan', 12: 'Somalia', 13: 'Tunisia', 14: 'Morocco', 15: 'Syria', 16: 'United_Arab_Emirates', 17: 'Algeria', 18: 'Egypt', 19: 'Qatar', 20: 'Libya'}

OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 13.40 %
MACRO AVERAGE RECALL SCORE: 11.00 %
MACRO AVERAGE F1 SCORE: 9.85 %
OVERALL ACCURACY: 14.55 %



# task2

In [76]:
import random
from tqdm import tqdm_notebook
train_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/train_labeled.tsv','\t')
g = train_df['#4 province_label'].value_counts()[0]
for i in tqdm_notebook(train_df['#4 province_label'].unique()):
    filter_df = train_df[train_df['#4 province_label'] == i]
    count = len(filter_df)
    while (count<=g):
        count+=1
        idx = random.randint(0,len(filter_df)-1)
        train_df = pd.concat([train_df,filter_df[idx:idx+1]])
train_df['#4 province_label'].value_counts()
train_df.to_csv('../tsv/oversample_train2.tsv')

/home/nikamanth/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


# unbalanced

In [9]:
train_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/train_labeled.tsv',sep='\t')
dev_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv',sep='\t')

X_train_original,y_train_original = train_df["#2 tweet_content"],train_df["#4 province_label"]
X_dev_original,y_dev_original = dev_df["#2 tweet_content"],dev_df["#4 province_label"]

In [10]:
labels = {}
uni = y_train_original.unique()
for i in range(len(uni)):
    labels[uni[i]] = i

y_train_index = [labels[i] for i in y_train_original]
y_dev_index = [labels[i] for i in y_dev_original]

y_train = to_categorical(y_train_index, num_classes=100)
y_dev = to_categorical(y_dev_index, num_classes=100)

In [11]:
X_train_corrected, X_dev_corrected = preprocess_text(X_train_original, X_dev_original)
X_train = bert.encode(X_train_corrected,batch_size=32,show_progress_bar=True)
X_dev = bert.encode(X_dev_corrected,batch_size=32,show_progress_bar=True)
X_train = np.vstack(X_train)
X_dev = np.vstack(X_dev)

Batches:   0%|          | 0/657 [00:00<?, ?it/s]

Batches: 100%|██████████| 155/155 [00:57<00:00,  2.68it/s]


In [12]:
model = Sequential()
model.add(Dense(512, activation='relu'))
model.add(Dense(100, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=50,validation_data=(X_dev,y_dev),
                   callbacks=[callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                                      patience=10, verbose=0, mode='auto', 
                                                      baseline=None, restore_best_weights=True)])
prediction = model.predict_classes(X_dev)

Train on 21000 samples, validate on 4957 samples
Epoch 1/50
21000/21000 [==============================] - 3s 143us/step - loss: 4.4520 - accuracy: 0.0488 - val_loss: 4.4893 - val_accuracy: 0.0317
Epoch 2/50
21000/21000 [==============================] - 3s 123us/step - loss: 4.1630 - accuracy: 0.0902 - val_loss: 4.4787 - val_accuracy: 0.0353
Epoch 3/50
21000/21000 [==============================] - 3s 125us/step - loss: 3.9883 - accuracy: 0.1178 - val_loss: 4.4951 - val_accuracy: 0.0363
Epoch 4/50
21000/21000 [==============================] - 3s 135us/step - loss: 3.8439 - accuracy: 0.1409 - val_loss: 4.5380 - val_accuracy: 0.0369
Epoch 5/50
21000/21000 [==============================] - 3s 132us/step - loss: 3.7154 - accuracy: 0.1609 - val_loss: 4.5986 - val_accuracy: 0.0341
Epoch 6/50
21000/21000 [==============================] - 3s 128us/step - loss: 3.5923 - accuracy: 0.1830 - val_loss: 4.6482 - val_accuracy: 0.0359
Epoch 7/50
21000/21000 [==============================] - 3s 12

In [13]:
reverse_label_map = {value : key for (key, value) in labels.items()}
print(reverse_label_map)
pred = model.predict_classes(X_dev)
pred = [reverse_label_map[i] for i in pred]
# print(pred)
len(pred)
write_result2('bert_task2_unbal.txt',pred)

{0: 'iq_Al-Anbar', 1: 'eg_Alexandria', 2: 'iq_Maysan', 3: 'ma_Oriental', 4: 'ly_Al-Jabal-al-Akhdar', 5: 'ae_Fujairah', 6: 'eg_Ismailia', 7: 'iq_Baghdad', 8: 'eg_Dakahlia', 9: 'mr_Nouakchott', 10: 'eg_Qena', 11: 'ma_Marrakech-Tensift-Al-Haouz', 12: 'sa_Tabuk', 13: 'eg_Asyut', 14: 'iq_Karbala', 15: 'bh_Capital', 16: 'sy_Damascus-City', 17: 'sa_Najran', 18: 'dj_Djibouti', 19: 'lb_Akkar', 20: 'om_Musandam', 21: 'ps_Gaza-Strip', 22: 'dz_Oran', 23: 'so_Banaadir', 24: 'sy_As-Suwayda', 25: 'eg_Faiyum', 26: 'jo_Aqaba', 27: 'eg_Cairo', 28: 'lb_North-Lebanon', 29: 'eg_Port-Said', 30: 'eg_Monufia', 31: 'tn_Sousse', 32: 'eg_Beheira', 33: 'sa_Ash-Sharqiyah', 34: 'eg_Gharbia', 35: 'ae_Ras-Al-Khaymah', 36: 'eg_Minya', 37: 'om_Al-Batnah', 38: 'kw_Jahra', 39: 'dz_Jijel', 40: 'dz_Béchar', 41: 'eg_Sohag', 42: 'sy_Hims', 43: 'iq_An-Najaf', 44: 'lb_South-Lebanon', 45: 'eg_Aswan', 46: 'ye_Ibb', 47: 'iq_Dihok', 48: 'sy_Aleppo', 49: 'iq_Al-Muthannia', 50: 'dz_Bordj-Bou-Arreridj\u200e', 51: 'ly_Tripoli', 52: 'y

# balanced

In [14]:
train_df = pd.read_csv('../tsv/oversample_train2.tsv')
dev_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv',sep='\t')

X_train_original,y_train_original = train_df["#2 tweet_content"],train_df["#4 province_label"]
X_dev_original,y_dev_original = dev_df["#2 tweet_content"],dev_df["#4 province_label"]

In [15]:
labels = {}
uni = y_train_original.unique()
for i in range(len(uni)):
    labels[uni[i]] = i

y_train_index = [labels[i] for i in y_train_original]
y_dev_index = [labels[i] for i in y_dev_original]

y_train = to_categorical(y_train_index, num_classes=100)
y_dev = to_categorical(y_dev_index, num_classes=100)

In [16]:
X_train_corrected, X_dev_corrected = preprocess_text(X_train_original, X_dev_original)
X_train = bert.encode(X_train_corrected,batch_size=32,show_progress_bar=True)
X_dev = bert.encode(X_dev_corrected,batch_size=32,show_progress_bar=True)
X_train = np.vstack(X_train)
X_dev = np.vstack(X_dev)

Batches:   0%|          | 0/1238 [00:00<?, ?it/s]

Batches: 100%|██████████| 155/155 [01:01<00:00,  2.52it/s]


In [17]:
model = Sequential()
model.add(Dense(512, activation='relu'))
model.add(Dense(100, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=50,validation_data=(X_dev,y_dev),
                   callbacks=[callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
                                                      patience=10, verbose=0, mode='auto', 
                                                      baseline=None, restore_best_weights=True)])
prediction = model.predict_classes(X_dev)

Train on 39600 samples, validate on 4957 samples
Epoch 1/50
39600/39600 [==============================] - 5s 125us/step - loss: 4.2835 - accuracy: 0.0838 - val_loss: 4.5187 - val_accuracy: 0.0297
Epoch 2/50
39600/39600 [==============================] - 5s 119us/step - loss: 3.8098 - accuracy: 0.1594 - val_loss: 4.6084 - val_accuracy: 0.0311
Epoch 3/50
39600/39600 [==============================] - 5s 119us/step - loss: 3.5040 - accuracy: 0.2179 - val_loss: 4.7482 - val_accuracy: 0.0323
Epoch 4/50
39600/39600 [==============================] - 5s 121us/step - loss: 3.2453 - accuracy: 0.2734 - val_loss: 4.8741 - val_accuracy: 0.0341
Epoch 5/50
39600/39600 [==============================] - 5s 120us/step - loss: 3.0085 - accuracy: 0.3277 - val_loss: 5.0202 - val_accuracy: 0.0331
Epoch 6/50
39600/39600 [==============================] - 5s 122us/step - loss: 2.7892 - accuracy: 0.3763 - val_loss: 5.1588 - val_accuracy: 0.0307
Epoch 7/50
39600/39600 [==============================] - 5s 12

In [18]:
reverse_label_map = {value : key for (key, value) in labels.items()}
print(reverse_label_map)
pred = model.predict_classes(X_dev)
pred = [reverse_label_map[i] for i in pred]
# print(pred)
len(pred)
write_result2('bert_task2_unbal.txt',pred)

{0: 'iq_Al-Anbar', 1: 'eg_Alexandria', 2: 'iq_Maysan', 3: 'ma_Oriental', 4: 'ly_Al-Jabal-al-Akhdar', 5: 'ae_Fujairah', 6: 'eg_Ismailia', 7: 'iq_Baghdad', 8: 'eg_Dakahlia', 9: 'mr_Nouakchott', 10: 'eg_Qena', 11: 'ma_Marrakech-Tensift-Al-Haouz', 12: 'sa_Tabuk', 13: 'eg_Asyut', 14: 'iq_Karbala', 15: 'bh_Capital', 16: 'sy_Damascus-City', 17: 'sa_Najran', 18: 'dj_Djibouti', 19: 'lb_Akkar', 20: 'om_Musandam', 21: 'ps_Gaza-Strip', 22: 'dz_Oran', 23: 'so_Banaadir', 24: 'sy_As-Suwayda', 25: 'eg_Faiyum', 26: 'jo_Aqaba', 27: 'eg_Cairo', 28: 'lb_North-Lebanon', 29: 'eg_Port-Said', 30: 'eg_Monufia', 31: 'tn_Sousse', 32: 'eg_Beheira', 33: 'sa_Ash-Sharqiyah', 34: 'eg_Gharbia', 35: 'ae_Ras-Al-Khaymah', 36: 'eg_Minya', 37: 'om_Al-Batnah', 38: 'kw_Jahra', 39: 'dz_Jijel', 40: 'dz_Béchar', 41: 'eg_Sohag', 42: 'sy_Hims', 43: 'iq_An-Najaf', 44: 'lb_South-Lebanon', 45: 'eg_Aswan', 46: 'ye_Ibb', 47: 'iq_Dihok', 48: 'sy_Aleppo', 49: 'iq_Al-Muthannia', 50: 'dz_Bordj-Bou-Arreridj\u200e', 51: 'ly_Tripoli', 52: 'y